In [1]:
import glob as glob
import os
from pathlib import Path
import re
from collections import defaultdict

In [2]:
shopping_list_files = sorted(Path('input').glob('**/*'))
combined = []
for shopping_list_file in shopping_list_files:
    with open(shopping_list_file) as f:
        shopping_list = [line.strip().lower() for line in f.readlines()]
    person = shopping_list_file.stem.capitalize()
    initial = person[0]
    combined += ((el, initial) for el in shopping_list)
    print(person)
    print('-'*10)
    for item in shopping_list:
        print(item)
    print()

Nana
----------
tin coco
salad cream
2 bread
1 cheap bread for birds
frozen aunt bessie yorkshire's small ones
cheap no sugar lemonade
quorn mince
book 2nd class stamps

Tina
----------
2 pints milk
2 packs quorn slices
2 pk ham
1 dairy free butter
anchor butter
low fat cheese triangles
small pk mushrooms
broccoli
8 bananas
bag apples
oranges
carrots
2 avocados
1 tomatoes on the vine
lettuce
strawberries
2 soya milk
crisps
2 bread

Val
----------
sugar
chocolate hobnobs
fruit corner yogurts
1 rugged white loaf
red grapes
cooking apples
2 peppers
small bag of onions
1  milk



In [3]:
category_files = sorted(Path('categories').glob('**/*'))
category_to_items = {}
for category_file in category_files:
    category = category_file.stem.lower()
    with open(category_file) as f:
        items = {line.strip().lower():True for line in f.readlines()}
        keys = list(items.keys())
        for item in keys:
            items[f'{item}s'] = True
        category_to_items[category] = items
        
def remove_digits(item):
    return ''.join([i for i in item if not i.isdigit()])

def remove_parentheses(item):
    return re.sub("[\(\[].*?[\)\]]", "", item)

def remove_multiple_space(item):
    return " ".join(item.split())

def clean_item(item):
    return remove_multiple_space(remove_digits(remove_parentheses(item)).strip().lower())

def search_categories(s):
    for category, items in category_to_items.items():
            if s in items:
                return category
            
def get_item_category(item):
    category = search_categories(item)
    if category:
        return category
    tokens = item.split()    
    for token in tokens:
        category = search_categories(token)
        if category:
            return category
        
d = defaultdict(list)
for item in combined:
    cleaned_item = clean_item(item[0])
    category = get_item_category(cleaned_item)
    if not category:
        print(cleaned_item)
    d[category].append(item)

In [6]:
with open('category_order.txt') as f:
    category_order = [el.strip().lower() for el in f.readlines()]

sorted_items = []
for category in category_order:
    if category in d:
        for item in d[category]:
            sorted_items.append(item)
            print('   '.join(item))
            
assert len(combined) == len(sorted_items)

book 2nd class stamps   N
small pk mushrooms   T
broccoli   T
8 bananas   T
bag apples   T
oranges   T
carrots   T
2 avocados   T
1 tomatoes on the vine   T
lettuce   T
strawberries   T
red grapes   V
cooking apples   V
2 peppers   V
small bag of onions   V
2 bread   N
1 cheap bread for birds   N
2 bread   T
1 rugged white loaf   V
2 pints milk   T
1 dairy free butter   T
anchor butter   T
low fat cheese triangles   T
2 soya milk   T
fruit corner yogurts   V
1  milk   V
2 pk ham   T
quorn mince   N
2 packs quorn slices   T
salad cream   N
crisps   T
chocolate hobnobs   V
cheap no sugar lemonade   N
sugar   V
tin coco   N
frozen aunt bessie yorkshire's small ones   N
